In [1]:
import numpy
import random
import tensorflow
import gymnasium as gym

from collections import deque
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import RMSprop

In [4]:
def Brain(input_shape, action_shape):
    _input = Input(shape=input_shape)
    _layer = Dense(512, activation='relu', kernel_initializer='he_uniform')(_input)
    _layer = Dense(256, activation='relu', kernel_initializer='he_uniform')(_layer)
    _layer = Dense(64, activation='relu', kernel_initializer='he_uniform')(_layer)
    _layer = Dense(action_shape, activation='linear', kernel_initializer='he_uniform')(_layer)
    _model = Model(inputs=_input, outputs=_layer)
    # Note: Optimizer and compile are not strictly needed for inference,
    # but load_model might expect them depending on how the model was saved.
    # Including them here for compatibility if the saved model includes optimizer state.
    _optimizer = RMSprop(learning_rate=0.00025, rho=0.95, epsilon=0.01)
    _model.compile(loss='mse', optimizer=_optimizer, metrics=['accuracy'])
    return _model

In [5]:
@tensorflow.keras.utils.register_keras_serializable(package='custom', name='mse')
def mse(y_true, y_pred):
    return tensorflow.keras.losses.mean_squared_error(y_true, y_pred)

In [8]:

class DQNAgent:

    def __init__(self):
        # Path to the saved model file
        self.model_name = 'models/cartpole-dqn.h5'

        # Environment setup (needed for state/action sizes and running the simulation)
        self.env = gym.make('CartPole-v1', render_mode='human') # Use human render mode to visualize
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n

        # Model will be loaded in the run method
        self.model = None

    def run(self):
        # Load the trained model
        try:
            # Pass custom_objects to load the custom mse function
            self.model = load_model(self.model_name, custom_objects={'mse': mse})
            print(f"Model loaded successfully from {self.model_name}")
        except Exception as e:
            print(f"Error loading model: {e}")
            print("Please ensure the model has been trained and saved to the correct path.")
            return

        print("Running the trained agent...")
        for i in range(25): # Run for 25 episodes
            _state, _ = self.env.reset()
            _done = False
            _lifetime = 0

            while not _done:
                # The environment rendering is handled by render_mode='human'
                # self.env.render() # This line is not needed with render_mode='human'

                # Reshape state for model prediction
                _state = numpy.reshape(_state, [1, self.state_size])

                # Predict the best action using the loaded model
                _action = numpy.argmax(self.model.predict(_state, verbose=0)) # verbose=0 to reduce output

                # Take the action in the environment
                _nxt_state, _reward, terminated, truncated, _info = self.env.step(_action)
                _done = terminated or truncated

                # Update state
                _state = _nxt_state

                _lifetime = _lifetime + 1

                if _done:
                    # Show score and stop the game
                    msg = 'Episode: {}, score: {}'
                    print(msg.format(i + 1, _lifetime)) # Use i + 1 for 1-based episode numbering
                    break

        self.env.close() # Close the environment window after running

In [ ]:
agent = DQNAgent()
agent.run()

Model loaded successfully from models/cartpole-dqn.h5
Running the trained agent...
Episode: 1, score: 449
Episode: 2, score: 451
Episode: 3, score: 467
Episode: 4, score: 405
Episode: 5, score: 451
Episode: 6, score: 404
Episode: 7, score: 441
Episode: 8, score: 434
Episode: 9, score: 425
Episode: 10, score: 408
Episode: 11, score: 435
Episode: 12, score: 423
Episode: 13, score: 427
Episode: 14, score: 410
Episode: 15, score: 424
Episode: 16, score: 431
Episode: 17, score: 455
Episode: 18, score: 451
Episode: 19, score: 433
Episode: 20, score: 443
Episode: 21, score: 438
Episode: 22, score: 419
Episode: 23, score: 437
Episode: 24, score: 422
Episode: 25, score: 412


: 